## Logging in python

### **1. Cos'è il logging?**

Il logging è un sistema strutturato per registrare eventi che accadono durante il run di un programma.
- Non è un semplice print() e non solo dei semplici errori.

Ogni volta che succede qualcosa di rilevante, il programma lascia una traccia scritta da qualche parte (file, console, server…).

Ogni messaggio di log riporta cosa è successo, quando, dove, dove viene salvato (se si salva) e quanto è grave. Ci sono diversi livelli di gravità:
- `DEBUG` → dettagli tecnici
- `INFO` → cose normali ma importanti
- `WARNING` → qualcosa di strano
- `ERROR` → qualcosa è andato male
- `CRITICAL` → il programma sta per morire

### **2. I 4 pilastri del logging**

#### 1. **LOGGER**
Il logger è l’oggetto che emette i messaggi

In [ ]:
import logging
logger = logging.getLogger("mio_modulo")
logger.info("Messaggio")

- Ha un nome ("accise", "myapp.db", ecc.)
- Ha un livello minimo
- Non scrive da solo: passa i messaggi agli handler

Importante:
- `logging.info()` usa il root logger
- `logging.getLogger("nome")` crea un logger tuo

#### 2. **LEVEL** (livelli):
I livelli indicano quanto è grave un messaggio.

| Livello  | Numero | Quando usarlo               |
| -------- | ------ | --------------------------- |
| DEBUG    | 10     | Dettagli interni (sviluppo) |
| INFO     | 20     | Stato normale               |
| WARNING  | 30     | Qualcosa di strano          |
| ERROR    | 40     | Errore gestito              |
| CRITICAL | 50     | Crash serio                 |

Un logger mostra solo i messaggi ≥ al suo livello.
Se il livello è INFO: DEBUG non lo mostra, ma mostra INFO, ERROR e CRITICAL

#### 3. **HANDLER**
Gli handler decidono DOVE va il log

Un logger può avere più handler, per esempio:
- `StreamHandler` → console
- `FileHandler` → file
- `RotatingFileHandler` → file con limite

#### 4. **FORMATTER**

Decide COME appare il messaggio

| Placeholder   | Significato |
| ------------- | ----------- |
| %(asctime)s   | Data/ora    |
| %(levelname)s | Livello     |
| %(name)s      | Nome logger |
| %(message)s   | Messaggio   |
| %(filename)s  | File        |
| %(lineno)d    | Riga        |


### **3. Flusso del logging**

In [ ]:
logger.error("Errore!")

Quando si avvia un programma, il logging passa in questo flusso,
e se uno di questi livelli si blocca, il log non passa

### **4. Root logger vs Logger nominato**

In [ ]:
#ROOT LOGGER

import logging
logging.info("ciao")

- Usa il logger globale
- Poco controllabile
- `basicConfig` agisce su lui

In [ ]:
#LOGGER NOMINATO

import logging
logger = logging.getLogger("mio_modulo")
logger.info("Messaggio informativo")

Il Logger nominato è un logger con un nome spicifico, tipicamente legatp ad un modulo o funzione con cui si sta lavorando. Si possono avere più logger a disposizione, ogniuno con i suoi livelli e handler.

I logger nominati per default passano i messaggi al root logger se non hanno i handler prorpi, di conseguenza per non farli finire sul root bisogna dotarli di handelr propri.

### **5. Creare un logger**

#### 1. Logger semplice

In [ ]:
import logging

# Crea un logger nominato
logger = logging.getLogger("nome_logger")  # Nome a tua scelta
logger.setLevel(logging.INFO)              # Livello minimo dei log

# Crea un formatter semplice
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

# Crea un handler per la console
console_handler = logging.StreamHandler()  # Scrive su console
console_handler.setFormatter(formatter)    # Viene associato il il formatter
console_handler.setLevel(logging.INFO)     # Viene associato il minimo per questo handler, ovvero INFO

# Aggiunge l'handler al logger
logger.addHandler(console_handler)

# Test del logger
logger.debug("DEBUG: messaggio dettagliato")  # Non apparirà (perché stato impostato il livello INFO)
logger.info("INFO: messaggio informativo")    # Apparirà
logger.warning("WARNING: attenzione!")        # Apparirà

#### 2. Logger con file

In [ ]:
formatter_file = logging.Formatter(
    "%(asctime)s | %(levelname)s | %(name)s | %(filename)s:%(lineno)d | %(message)s"
)

# Crea un handler per il file
file_handler = logging.FileHandler("app.log", encoding="utf-8")     # Viene usato FileHandeler al posto di StreamHandelr
file_handler.setFormatter(logging.Formatter(formatter_file))
console_handler.setLevel(logging.INFO)

# Aggiunge l'handler
logger.addHandler(file_handler)

# Con il codice precedente e questa aggiunta i log appariranno sia su console che su file
logger.info("Questo messaggio va su console e su file!")

In [ ]:
# Logger globale
logger.setLevel(logging.DEBUG)  # prende tutto da DEBUG in su

# Handler console
console_handler.setLevel(logging.INFO)  # solo INFO, WARNING, ERROR, CRITICAL

# Handler file
file_handler.setLevel(logging.DEBUG)    # tutto, anche DEBUG

logger.debug("DEBUG: andrà solo sul file")
logger.info("INFO: andrà su console e file")


#### 3. Logger con rotazione dei file (utile in produzione)

Permette di non avere file enormi con logging vecchi, raggiunta una quota fissata di dimensione viene creato un nuovo log file

In [ ]:
from logging.handlers import RotatingFileHandler

formatter_file = logging.Formatter(
    "%(asctime)s | %(levelname)s | %(name)s | %(filename)s:%(lineno)d | %(message)s"
)

# Definizione delle caratteristiche della rotazione dei file
rotating_handler = RotatingFileHandler(
    "app_rotating.log",  # nome file
    maxBytes=2_000_000,  # 2 MB
    backupCount=5,       # massimo 5 file di backup
    encoding="utf-8"
)

rotating_handler.setFormatter(logging.Formatter(formatter_file))
rotating_handler.setLevel(logging.INFO)
logger.addHandler(rotating_handler)

logger.info("Messaggio che va anche sul file rotante")

#### 4. Silenziare le  librerie rumorose.

Molte librerie esterne possono scrivere diversi messaggi di log, anche informativi o debug, per avere un log pulito e ordinato si possono silenziare.

In [ ]:
# Ottiene il logger specfico di una determinata libreria, e importa il livello minimo, in questo caso a WARNING
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

### **6. Gestire le eccezioni**

In python non tutte le eccezioni vengono catturate con il `try`/`except`. Se una eccezione sfugge, normalemnte non viene catturate e di conseguenza il traceback non viene stampato sulla console e si chiude il programma.

In [ ]:
import sys

# Serve per accedere a sys.__excepthook__ (gestore originale python),
# il meccanismo interno di Pyhton per gestire le eccezzioni non catturare

In [ ]:
def handle_uncaught(
        exc_type,       # tipo di eccezione, es. ValueError
        exc_value,      # il messaggio dell'eccezione
        exc_traceback   # lo stack comlpeto del punto in cui è accaduto
        ):

    if issubclass(exc_type, KeyboardInterrupt):                     # Eccezione KeyboardInterrupt: quando si fa Ctrl+C per stoppare
        sys.__excepthook__(exc_type, exc_value, exc_traceback)      # Non viene loggata e ritorna vuoto
        return
    # Tutte le altre eccezzioni "sfuggite" vengono loggate come CRITICAL
    logger.critical("Eccezione non gestita",
                    exc_info=(exc_type, exc_value, exc_traceback))  # exc_info include tipo, messaggio e tracceback

# Richiamo della funzione
sys.excepthook = handle_uncaught    # Ogni volta che succede un eccezione non catturata si avvia la funzione


Perché è utile:
- intercetta crach
- logga eccezioni critiche
- mantiene un comportamento nautrale per Ctrl+C, il programma si può interrompere senza confusione

### **7. Schema**

In [ ]:
# 1. Creo logger
logger = logging.getLogger("mio_app")
logger.setLevel(logging.DEBUG)

# 2. Creo formatter
fmt_console = logging.Formatter("[%(levelname)s] %(message)s")
fmt_file = logging.Formatter("%(asctime)s | %(levelname)s | %(filename)s:%(lineno)d | %(message)s")

# 3. Creo handler console
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(fmt_console)

# 4. Creo handler file
fh = RotatingFileHandler("app.log", maxBytes=2_000_000, backupCount=5, encoding="utf-8")
fh.setLevel(logging.DEBUG)
fh.setFormatter(fmt_file)

# 5. Aggiungo handler al logger
logger.addHandler(ch)
logger.addHandler(fh)

# 6. Log di test
logger.debug("DEBUG → file")
logger.info("INFO → console + file")
logger.warning("WARNING → console + file")
logger.error("ERROR → console + file")


### **8. Implementazione di logging in un progetto**

In un progetto comlpesso è fondamentale inserire un logging dettagliato e preciso. Come già anticipato nelle precedenti sezioni si possono creare più logger ogniuno con il suo handler, per esempio si possono crearne uno per ogni punto fondamentale del progetto, per ogni modulo,...

Se il progetto prevede diverse sezioni si possono implementare logger diversi per sezioni diverse.

è consigliato creare un `logger_utils.py` di set-up per il logger e uno per il set-up di tutto il processo:

In [ ]:
import logging
from logging.handlers import RotatingFileHandler
import sys
from pathlib import Path

In [ ]:
def setup_logger (
    name:str,                     # nome del logger
    level=logging.INFO,           # livello minimo
    log_to_file=False,            # scrivere su file?
    new_file = False,
    log_file="app.log",           # nome file
    max_bytes=2_000_000,          # dimensione massima file
    backup_count=5                # numero backup file rotanti
) -> logging.Logger:
    """
    Crea e restituisce un logger stabile con console e (opzionale) file.
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.propagate = False    # evita doppio logging dal root logger

    # 1. Evita duplicati se logger già configurato
    if logger.handlers():
        return logger

    # 2. FORMATTER: sia per console che per file. Per console è ridotto
    fmt_console = logging.Formatter("[%(levelname)s] %(message)s")
    fmt_file = logging.Formatter("%(asctime)s | %(levelname)s | %(name)s | %(filename)s:%(lineno)d | %(message)s")

    # 3.1. HANDLER CONSOLE
    console_handler = logging.StreamHandler()   # di default, se non esplicitato manda i log su sys.stderr
                                                # sys.stdout: è un output normale. Si usa quando si vuole che i log vengano trattati coome output normali.
                                                # sys.stderr: errori, avvisi, log di errore. SI usa quando si vuole separare gli output con i logs
    console_handler.setLevel(level)
    console_handler.setFormatter(fmt_console)
    logger.addHandler(console_handler)
    # 3.2. HANDLER FILE (opzionale)
    if log_to_file:
        log_path = Path(log_file)
        log_path.parent.mkdir(parents=True, exist_ok=True)
        if new_file:                            # Viene impostato su true ogni volta cancella il file vecchio per farle uno nuovo e pulito
            if log_path.exists():
                log_path.unlink()  # cancella il file esistente
        file_handler = RotatingFileHandler(log_file, maxBytes=max_bytes, backupCount=backup_count,encoding="utf-8")
        file_handler.setLevel(level)
        file_handler.setFormatter(fmt_file)
        logger.addHandler(file_handler)

    return logger

In [ ]:
def handle_uncaught(exc_type, exc_value, exc_tb, logger=None):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_tb)
        return
    if logger is None:
        logger = logging.getLogger("uncaught")
    logger.critical("Eccezione non gestita", exc_info=(exc_type, exc_value, exc_tb))

Nel file `log_setup.py` viene instanziato il il `setup_logger` per creare il logger determinato per una singola funzione/modulo.

In [ ]:
LOGS_DIR = Path('Logs')
LOGS_DIR.mkdir(exist_ok=True)

# Logger funzione uno
logger_funzione_uno = setup_logger(
    name="funzione_uno",
    level=logging.INFO,
    log_to_file=True,
    log_file=LOGS_DIR / "funzione_uno/funzione_uno.log"
)

sys.excepthook = lambda exc_type, exc_value, exc_tb: handle_uncaught(
    exc_type, exc_value, exc_tb, logger=logger_funzione_uno
)
